<a href="https://colab.research.google.com/github/SHANIQUETS/Healthcare-Integration-Pipeline/blob/main/integrated_data_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  1.  Integrating Patient Data for Targeted Healthcare Insight: A Data Pipeline for Demographics, Conditions, and Medication Trends

## Problem Statement:

In healthcare, data is often spread across different systems and formats, making it tough to get a clear, unified view of patient information. Without pulling together demographics, health conditions, and medication trends in one place, it’s hard to identify which groups are at higher risk, allocate resources efficiently, or set up preventive care where it’s needed. Real-time data is also limited, which means delayed responses for conditions that might need immediate action.

This project focuses on building a data pipeline to bring together structured and semi-structured patient data from various sources, including real-time streams. By integrating this data into both relational and NoSQL databases, the pipeline will enable detailed reporting on patient demographics, conditions, and medication trends. The goal is to give healthcare providers easy access to insights that support smarter, data-driven decisions for better patient care and proactive health management.

In [1]:
# Import the required libraries
import pandas as pd
from sqlalchemy import create_engine
import json

## 2. Data Processing
##### Load the necessary files into DataFrames

In [2]:
# Load data

url=("https://raw.githubusercontent.com/SHANIQUETS/Healthcare-Integration-Pipeline/refs/heads/main/conditions.csv?token=GHSAT0AAAAAACZTEX5D2HBIENWFZJKWYXX2ZY66S2A")
conditions_data = pd.read_csv(url)

url=("https://raw.githubusercontent.com/SHANIQUETS/Healthcare-Integration-Pipeline/refs/heads/main/encounters.csv?token=GHSAT0AAAAAACZTEX5DYU63FGYNK3B5ZFOSZY66TSA")
encounters_data = pd.read_csv(url)

url=("https://raw.githubusercontent.com/SHANIQUETS/Healthcare-Integration-Pipeline/refs/heads/main/medications.csv?token=GHSAT0AAAAAACZTEX5DHWSVG334OFLQR6QUZY66UDA")
medication_data = pd.read_csv(url)

url=("https://raw.githubusercontent.com/SHANIQUETS/Healthcare-Integration-Pipeline/refs/heads/main/patients.csv?token=GHSAT0AAAAAACZTEX5DIARSGOIU5KGS2BXMZY66UQQ")
patients_data = pd.read_csv(url)

### 3. Clean and Transform Data



#### Clean Conditions Data

In [3]:
# Inspect the data
conditions_data.head()
conditions_data.info()  # to check data types and missing values
conditions_data.isnull().sum()  # check for missing values in each column

# Fill missing values in the STOP column with "Unknown"
conditions_data['STOP'] = conditions_data['STOP'].fillna('Unknown')

# Convert START column to datetime
conditions_data['START'] = pd.to_datetime(conditions_data['START'], format="%m/%d/%Y", errors='coerce')

# Convert STOP to datetime where it’s not "Unknown"; leave "Unknown" as is
conditions_data['STOP'] = pd.to_datetime(
    conditions_data['STOP'].replace("Unknown", pd.NaT),  # Replace "Unknown" with NaT for datetime compatibility
    format="%m/%d/%Y",
    errors='coerce'
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8376 entries, 0 to 8375
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   START        8376 non-null   object
 1   STOP         4565 non-null   object
 2   PATIENT      8376 non-null   object
 3   ENCOUNTER    8376 non-null   object
 4   CODE         8376 non-null   int64 
 5   DESCRIPTION  8376 non-null   object
dtypes: int64(1), object(5)
memory usage: 392.8+ KB


In [4]:
# Display unique values in START and STOP columns to diagnose
print("Unique values in START column:")
print(conditions_data['START'].unique())

print("\nUnique values in STOP column:")
print(conditions_data['STOP'].unique())

Unique values in START column:
<DatetimeArray>
['NaT']
Length: 1, dtype: datetime64[ns]

Unique values in STOP column:
<DatetimeArray>
['NaT']
Length: 1, dtype: datetime64[ns]


In [ ]:
# Check the current data types
print(conditions_data.dtypes)

START          datetime64[ns]
STOP           datetime64[ns]
PATIENT                object
ENCOUNTER              object
CODE                    int64
DESCRIPTION            object
dtype: object


In [5]:
# Step 1: Calculate DURATION_DAYS where both START and STOP are dates
conditions_data['DURATION_DAYS'] = (conditions_data['STOP'] - conditions_data['START']).dt.days

# Step 2: Extract the year from the START date
conditions_data['YEAR'] = conditions_data['START'].dt.year

# Display the updated DataFrame to confirm the new columns
print(conditions_data.head())

  START STOP                               PATIENT  \
0   NaT  NaT  1d604da9-9a81-4ba9-80c2-de3375d59b40   
1   NaT  NaT  8d4c4326-e9de-4f45-9a4c-f8c36bff89ae   
2   NaT  NaT  8d4c4326-e9de-4f45-9a4c-f8c36bff89ae   
3   NaT  NaT  10339b10-3cd1-4ac3-ac13-ec26728cb592   
4   NaT  NaT  f5dcd418-09fe-4a2f-baa0-3da800bd8c3a   

                              ENCOUNTER       CODE  \
0  8f104aa7-4ca9-4473-885a-bba2437df588   40055000   
1  9d35ec9f-352a-4629-92ef-38eae38437e7  444814009   
2  ae7555a9-eaff-4c09-98a7-21bc6ed1b1fd  195662009   
3  e1ab4933-07a1-49f0-b4bd-05500919061d   10509002   
4  b8f76eba-7795-4dcd-a544-f27ac2ef3d46  195662009   

                          DESCRIPTION  DURATION_DAYS  YEAR  
0        Chronic sinusitis (disorder)            NaN   NaN  
1          Viral sinusitis (disorder)            NaN   NaN  
2  Acute viral pharyngitis (disorder)            NaN   NaN  
3         Acute bronchitis (disorder)            NaN   NaN  
4  Acute viral pharyngitis (disorder)        

#### Clean Encounters Data

In [6]:
# Check for missing values
print(encounters_data.head())

                                     Id                 START  \
0  d0c40d10-8d87-447e-836e-99d26ad52ea5  2010-01-23T17:45:28Z   
1  e88bc3a9-007c-405e-aabc-792a38f4aa2b  2012-01-23T17:45:28Z   
2  8f104aa7-4ca9-4473-885a-bba2437df588  2001-05-01T15:02:18Z   
3  b85c339a-6076-43ed-b9d0-9cf013dec49d  2011-07-28T15:02:18Z   
4  dae2b7cb-1316-4b78-954f-fa610a6c6d0e  2010-07-27T12:58:08Z   

                   STOP                               PATIENT  \
0  2010-01-23T18:10:28Z  034e9e3b-2def-4559-bb2a-7850888ae060   
1  2012-01-23T18:00:28Z  034e9e3b-2def-4559-bb2a-7850888ae060   
2  2001-05-01T15:17:18Z  1d604da9-9a81-4ba9-80c2-de3375d59b40   
3  2011-07-28T15:17:18Z  1d604da9-9a81-4ba9-80c2-de3375d59b40   
4  2010-07-27T13:28:08Z  10339b10-3cd1-4ac3-ac13-ec26728cb592   

                           ORGANIZATION                              PROVIDER  \
0  e002090d-4e92-300e-b41e-7d1f21dee4c6  e6283e46-fd81-3611-9459-0edb1c3da357   
1  772ee193-bb9f-30eb-9939-21e86c8e4da5  6f1d59a7-a5bd-3

In [7]:
# Rename columns to remove spaces
encounters_data.columns = encounters_data.columns.str.replace(' ', '_', regex=False)

# Remove duplicates based on 'Id' column
encounters_data.drop_duplicates(subset='Id', keep='first', inplace=True)

# Convert 'START' and 'STOP' columns to datetime
encounters_data['START'] = pd.to_datetime(encounters_data['START'], errors='coerce')
encounters_data['STOP'] = pd.to_datetime(encounters_data['STOP'], errors='coerce')

# Convert numeric columns to appropriate types
numeric_columns = ['BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST', 'PAYER_COVERAGE']
for col in numeric_columns:
    encounters_data[col] = pd.to_numeric(encounters_data[col], errors='coerce')

# Handle missing values by filling with 0 or appropriate methods
encounters_data.fillna({
    'BASE_ENCOUNTER_COST': 0,
    'TOTAL_CLAIM_COST': 0,
    'PAYER_COVERAGE': 0,
    'REASONCODE': 'Unknown',  # Assuming 'Unknown' for unknown reasons
    'REASONDESCRIPTION': 'Not provided'
}, inplace=True)

In [8]:
# Display the cleaned data
print("Cleaned Encounters Data:")
print(encounters_data.head())

Cleaned Encounters Data:
                                     Id                     START  \
0  d0c40d10-8d87-447e-836e-99d26ad52ea5 2010-01-23 17:45:28+00:00   
1  e88bc3a9-007c-405e-aabc-792a38f4aa2b 2012-01-23 17:45:28+00:00   
2  8f104aa7-4ca9-4473-885a-bba2437df588 2001-05-01 15:02:18+00:00   
3  b85c339a-6076-43ed-b9d0-9cf013dec49d 2011-07-28 15:02:18+00:00   
4  dae2b7cb-1316-4b78-954f-fa610a6c6d0e 2010-07-27 12:58:08+00:00   

                       STOP                               PATIENT  \
0 2010-01-23 18:10:28+00:00  034e9e3b-2def-4559-bb2a-7850888ae060   
1 2012-01-23 18:00:28+00:00  034e9e3b-2def-4559-bb2a-7850888ae060   
2 2001-05-01 15:17:18+00:00  1d604da9-9a81-4ba9-80c2-de3375d59b40   
3 2011-07-28 15:17:18+00:00  1d604da9-9a81-4ba9-80c2-de3375d59b40   
4 2010-07-27 13:28:08+00:00  10339b10-3cd1-4ac3-ac13-ec26728cb592   

                           ORGANIZATION                              PROVIDER  \
0  e002090d-4e92-300e-b41e-7d1f21dee4c6  e6283e46-fd81-3611-9459

#### Clean  Medication Data

In [9]:
# Display the first few rows of the original data
print("Original Data:")
print(medication_data.head())

# Rename columns to remove spaces and make them more accessible
medication_data.columns = medication_data.columns.str.replace(' ', '_', regex=False)

# Remove duplicates based on 'PATIENT' and 'START' columns (assumed as unique identifiers)
medication_data.drop_duplicates(subset=['PATIENT', 'START'], keep='first', inplace=True)

# Convert 'START' and 'STOP' columns to datetime
medication_data['START'] = pd.to_datetime(medication_data['START'], errors='coerce')
medication_data['STOP'] = pd.to_datetime(medication_data['STOP'], errors='coerce')

# Convert numeric columns to appropriate types
numeric_columns = ['BASE_COST', 'PAYER_COVERAGE', 'DISPENSES', 'TOTALCOST']
for col in numeric_columns:
    medication_data[col] = pd.to_numeric(medication_data[col], errors='coerce')

# Handle missing values
medication_data.fillna({
    'BASE_COST': 0,
    'PAYER_COVERAGE': 0,
    'DISPENSES': 0,
    'TOTALCOST': 0,
    'REASONCODE': 'Unknown',  # Assuming 'Unknown' for unknown reasons
    'REASONDESCRIPTION': 'Not provided'  # Default description for missing reasons
}, inplace=True)

# Display the cleaned data
print("Cleaned Medication Data:")
print(medication_data.head())

Original Data:
                  START                  STOP  \
0  2010-05-05T00:26:23Z  2011-04-30T00:26:23Z   
1  2011-04-30T00:26:23Z  2012-04-24T00:26:23Z   
2  2012-04-24T00:26:23Z  2013-04-19T00:26:23Z   
3  2011-05-13T12:58:08Z  2011-05-27T12:58:08Z   
4  2011-12-08T15:02:18Z  2011-12-22T15:02:18Z   

                                PATIENT                                 PAYER  \
0  8d4c4326-e9de-4f45-9a4c-f8c36bff89ae  b1c428d6-4f07-31e0-90f0-68ffa6ff8c76   
1  8d4c4326-e9de-4f45-9a4c-f8c36bff89ae  b1c428d6-4f07-31e0-90f0-68ffa6ff8c76   
2  8d4c4326-e9de-4f45-9a4c-f8c36bff89ae  b1c428d6-4f07-31e0-90f0-68ffa6ff8c76   
3  10339b10-3cd1-4ac3-ac13-ec26728cb592  d47b3510-2895-3b70-9897-342d681c769d   
4  1d604da9-9a81-4ba9-80c2-de3375d59b40  b1c428d6-4f07-31e0-90f0-68ffa6ff8c76   

                              ENCOUNTER    CODE  \
0  1e0d6b0e-1711-4a25-99f9-b1c700c9b260  389221   
1  6aa37300-d1b4-48e7-a2f8-5e0f70f48f38  389221   
2  7253a9f9-6f6d-429a-926a-7b1d424eae3f  748856   

#### Clean Patients Data

In [10]:
import numpy as np
# Display the first few rows of the original data for review
print("Original Data:")
print(patients_data.head())

# 1. Rename columns to remove leading/trailing spaces and ensure consistent naming
patients_data.columns = [col.strip().lower() for col in patients_data.columns]

# 2. Handle missing values
# Replace empty strings with NaN
patients_data.replace('', np.nan, inplace=True)

# 3. Drop rows with critical missing values (e.g., BIRTHDATE or ID)
patients_data.dropna(subset=['birthdate', 'id'], inplace=True)

# 4. Convert date columns to datetime format
patients_data['birthdate'] = pd.to_datetime(patients_data['birthdate'], errors='coerce')
patients_data['deathdate'] = pd.to_datetime(patients_data['deathdate'], errors='coerce')

# 5. Clean name columns to remove non-ASCII characters and trailing spaces
patients_data['first'] = patients_data['first'].str.encode('ascii', 'ignore').str.decode('ascii').str.strip()
patients_data['last'] = patients_data['last'].str.encode('ascii', 'ignore').str.decode('ascii').str.strip()
patients_data['suffix'] = patients_data['suffix'].str.encode('ascii', 'ignore').str.decode('ascii').str.strip()

# 6. Normalise the case for string fields (e.g., names)
patients_data['first'] = patients_data['first'].str.title()
patients_data['last'] = patients_data['last'].str.title()

# 7. Ensure consistent formatting for address components
patients_data['address'] = patients_data['address'].str.strip()
patients_data['city'] = patients_data['city'].str.title()
patients_data['state'] = patients_data['state'].str.upper()  # Standardise to uppercase
patients_data['zip'] = patients_data['zip'].astype(str).str.zfill(5)  # Ensure ZIP codes are 5 digits

# 8. Check and remove duplicates based on ID
patients_data.drop_duplicates(subset=['id'], inplace=True)

# 9. Reset index after cleaning
patients_data.reset_index(drop=True, inplace=True)

# 10. Display the cleaned data
print("Cleaned Medication Data:")
print(patients_data.head())

Original Data:
                                     Id   BIRTHDATE DEATHDATE          SSN  \
0  1d604da9-9a81-4ba9-80c2-de3375d59b40  1989-05-25       NaN  999-76-6866   
1  034e9e3b-2def-4559-bb2a-7850888ae060  1983-11-14       NaN  999-73-5361   
2  10339b10-3cd1-4ac3-ac13-ec26728cb592  1992-06-02       NaN  999-27-3385   
3  8d4c4326-e9de-4f45-9a4c-f8c36bff89ae  1978-05-27       NaN  999-85-4926   
4  f5dcd418-09fe-4a2f-baa0-3da800bd8c3a  1996-10-18       NaN  999-60-7372   

     DRIVERS    PASSPORT PREFIX            FIRST           LAST SUFFIX  ...  \
0  S99984236  X19277260X    Mr.  José Eduardo181       Gómez206    NaN  ...   
1  S99962402  X88275464X    Mr.          Milo271        Feil794    NaN  ...   
2  S99972682  X73754411X    Mr.        Jayson808       Fadel536    NaN  ...   
3  S99974448  X40915583X   Mrs.       Mariana775  Rutherford999    NaN  ...   
4  S99915787  X86772962X    Mr.      Gregorio366         Auer97    NaN  ...   

                         BIRTHPLACE      

In [11]:
# 2. Drop rows with NaN in critical columns
patients_data.dropna(subset=['birthdate', 'id'], inplace=True)

# 3. Fill NaN values without using inplace=True to avoid warnings
# Keep deathdate as NaT for datetime
patients_data['deathdate'] = patients_data['deathdate'].fillna(pd.NaT)

# Fill healthcare_expenses with median
patients_data['healthcare_expenses'] = patients_data['healthcare_expenses'].fillna(patients_data['healthcare_expenses'].median())

# Fill healthcare_coverage with mean
patients_data['healthcare_coverage'] = patients_data['healthcare_coverage'].fillna(patients_data['healthcare_coverage'].mean())

# Fill categorical fields with a placeholder value
patients_data['city'] = patients_data['city'].fillna('Unknown')
patients_data['state'] = patients_data['state'].fillna('Unknown')

# For the other columns with NaN values, you can also decide on a strategy, for example:
patients_data['drivers'] = patients_data['drivers'].fillna('Unknown Driver')
patients_data['passport'] = patients_data['passport'].fillna('Unknown Passport')
patients_data['prefix'] = patients_data['prefix'].fillna('Mr./Mrs.')
patients_data['suffix'] = patients_data['suffix'].fillna('')
patients_data['maiden'] = patients_data['maiden'].fillna('Unknown Maiden')
patients_data['marital'] = patients_data['marital'].fillna('Unknown Marital Status')

# 4. Check for missing values after handling
print("Missing Values After Handling:")
print(patients_data.isnull().sum())

Missing Values After Handling:
id                        0
birthdate                 0
deathdate              1000
ssn                       0
drivers                   0
passport                  0
prefix                    0
first                     0
last                      0
suffix                    0
maiden                    0
marital                   0
race                      0
ethnicity                 0
gender                    0
birthplace                0
address                   0
city                      0
state                     0
county                    0
zip                       0
lat                       0
lon                       0
healthcare_expenses       0
healthcare_coverage       0
dtype: int64


### 4. Set Up ProgreSQL
###### set up a connection to PostgresSQL database

In [ ]:
engine = create_engine('postgresql:username:LondonSB14"@localhost:5432/mydatabase')

#load Dataframes into PostgreSQL
conditions_data.to_sql('conditions', engine, if_exist='replace', index=False)
encounters_data.to_sql('encounters', engine, if_exist='replace', index=False)
medication_data.to_sql('medication', engine, if_exist='replace', index=False)
patients_data.to_sql('patients', engine, if_exist='replace', index=False)

### 5. Set Up MongoDB
###### set up a connection to Mongo instance

In [ ]:
client = MongoClient('mongodb://localhost:27017')
db = client ['mydatabase']

In [ ]:
### Load Data into MongoDB

observations_collection = db ['observations']
observation_collection.insert_many(observation_data.to_dict('records'))

### 5. Create Data Mart
###### Create summarised tables for data marts.

In [ ]:
# Data Mart Creation

patients_